## Taller Regularización y SVM


Para esta tarea consideramos un conjunto de observaciones sobre una serie de variedades de vino tinto y blanco que implican sus propiedades químicas y su clasificación por parte de los catadores. La industria del vino ha experimentado un gran crecimiento en los últimos tiempos debido al aumento del consumo social. El precio del vino depende de un concepto bastante abstracto de apreciación del vino por parte de los catadores, cuya opinión puede tener un alto grado de variabilidad. El precio del vino depende en cierta medida de este factor tan volátil. Otro factor clave en la certificación y evaluación de la calidad del vino son las pruebas fisicoquímicas, que se realizan en laboratorio y tienen en cuenta factores como la acidez, el nivel de pH, la presencia de azúcar y otras propiedades químicas. Para el mercado del vino, sería interesante que la calidad humana de la cata pudiera relacionarse con las propiedades químicas del vino para que el proceso de certificación y evaluación de la calidad estuviera más controlado.

Se dispone de dos conjuntos de datos, uno de los cuales se refiere al vino tinto y cuenta con 1.599 variedades diferentes, y el otro, al vino blanco, con 4.898 variedades. Todos los vinos se producen en una zona concreta de Portugal. Se recogen datos sobre 12 propiedades diferentes de los vinos, una de las cuales es la calidad, basada en datos sensoriales, y el resto son propiedades químicas de los vinos, como la densidad, la acidez, el contenido de alcohol, etc. Todas las propiedades químicas de los vinos son variables continuas. La calidad es una variable ordinal con una clasificación posible de 1 (peor) a 10 (mejor). Cada variedad de vino es catada por tres catadores independientes y la clasificación final asignada es la mediana de la clasificación dada por los catadores.

Se espera que un modelo predictivo desarrollado a partir de estos datos sirva de orientación a los viñedos en cuanto a la calidad y el precio que se espera obtener de sus productos sin depender en gran medida de la volatilidad de los catadores.


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV


In [ ]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [ ]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
2065,7.8,0.27,0.28,1.8,0.050,21.0,127.0,0.99340,3.15,0.44,9.9,5,white
4031,6.4,0.29,0.57,1.0,0.060,15.0,120.0,0.99240,3.06,0.41,9.5,5,white
5972,7.5,0.77,0.20,8.1,0.098,30.0,92.0,0.99892,3.20,0.58,9.2,5,red
4581,6.0,0.27,0.26,1.3,0.038,32.0,138.0,0.99125,3.46,0.43,11.1,6,white
1124,9.6,0.23,0.40,1.5,0.044,19.0,135.0,0.99370,2.96,0.49,10.9,5,white


In [ ]:
data.quality.value_counts()

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

In [ ]:
data.type.value_counts()

white    4898
red      1599
Name: type, dtype: int64

In [ ]:
data.columns.values

array(['fixed acidity', 'volatile acidity', 'citric acid',
       'residual sugar', 'chlorides', 'free sulfur dioxide',
       'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',
       'quality', 'type', 'type2', 'quality2'], dtype=object)

# Ejercicio 1

Mostrar la tabla de frecuencias de la calidad por tipo de vino.

In [ ]:
# Tabla de frecuencias 1
pd.crosstab(index=data["type"], columns=[data["quality"]])

quality,3,4,5,6,7,8,9
type,,,,,,,
red,10,53,681,638,199,18,0
white,20,163,1457,2198,880,175,5


In [ ]:
# Tabla de frecuencias 2
pd.pivot_table(data, index = ["quality", "type"], aggfunc= np.sum)

alcohol  chlorides  ...  total sulfur dioxide  volatile acidity
quality type                            ...                                        
3       red       99.550000      1.225  ...                 249.0             8.845
        white    206.900000      1.086  ...                3412.0             6.665
4       red      544.050000      4.806  ...                1921.0            36.780
        white   1654.850000      8.166  ...               20420.5            62.140
5       red     6741.700000     63.153  ...               38486.0           392.965
        white  14291.480000     75.103  ...              219868.0           440.030
6       red     6781.633333     54.202  ...               26075.0           317.395
        white  23244.666667     99.388  ...              301230.0           572.720
7       red     2281.716667     15.241  ...                6969.0            80.380
        white  10003.783333     33.608  ...              110101.0           231.235
8       red      217.700000      1.232  ...                 602.0             7.620
        white   2036.300000      6.705  ...               22079.0            48.545
9       white     60.900000      0.137  ...                 580.0             1.490

[13 rows x 11 columns]

# Ejercicio 2

* Entrenar una regresión lineal para predecir la calidad del vino (Continuo)

* Analice los coeficientes

* Evaluar el RMSE

In [ ]:
# Crear variables binarias
data['type2'] = data['type'].map({'white':0, 'red':1})
data['quality2'] = data.quality.map({3:0, 4:0, 5:0, 6:1, 7:1, 8:1, 9:1})

In [ ]:
# Train-test

feature_cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide',
                 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol','type2']

X = data[feature_cols]
y = data.quality

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=456)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)  

(5197, 12) (1300, 12) (5197,) (1300,)


In [ ]:
#Modelo:

modelo1 = LinearRegression()
modelo1.fit(X_train, y_train)
y_pred = modelo1.predict(X_test)

print(modelo1.coef_)

### RMSE #####
print("RMSE: ",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

[ 8.54940888e-02 -1.49758834e+00 -6.78944004e-02  6.41440109e-02
 -7.78717492e-01  4.71921844e-03 -1.23848988e-03 -1.04642791e+02
  5.07676763e-01  6.92181980e-01  2.32039126e-01  3.84830323e-01]
RMSE:  0.7412790604053446


Analisis de los coeficientes:

1. fixed acidity: El aumento de una unidad de acidez fija implicaría un incremento de 0.085494 puntos en la calidad del vino.

2. volatile acidity: El aumento de una unidad en la acidez volátil implicaría una disminución de 1.4975 puntos en la calidad del vino.

3. citric acid: El aumento de una unidad en el ácido cítrico implicaría una disminución de 0.0678 puntos en la calidad del vino.

4. residual sugar: El aumento de una unidad del azúcar residual implicaría un incremento de 0.064144 puntos en la calidad del vino.

5. chlorides: El aumento de una unidad en las calorias implicaría una disminución de 0.7787 puntos en la calidad del vino

6. free sulfur dioxide: El aumento de una unidad del dióxido de azufre libre implicaría un incremento de 0.0431 puntos en la calidad del vino.

7. total sulfur dioxide: El aumento de una unidad del dióxido de azufre total implicaría una disminución de 1.2384 puntos en la calidad del vino

8. density: El aumento de una unidad de la densidad implicaría una disminución de 1.0464 puntos en la calidad del vino

9. pH: El aumento de una unidad del pH implicaría un incremento de 0.507 puntos en la calidad del vino.

10. sulphates: El aumento de una unidad del sulfato implicaría un incremento de 0.69218 puntos en la calidad del vino.

11. alcohol: El aumento de una unidad de alcohol implicaría un incremento de 0.2320 puntos en la calidad del vino.

12. type: Entre mas se acerque al tipo de vino rojo en una unidad, la calidad del vino aumenta en 0.3848


# Ejercicio 3

* Estimar una regresión de Ridge con alfa igual a 0,1 y 1.
* Compare y analice los coeficientes de Ridge con la regresión lineal
* Evaluar el RMSE

In [ ]:
#Regresión de Ridge con alpha = 0.1

ridgereg = Ridge(alpha=0.1, normalize=True)
ridgereg.fit(X_train, y_train)
y_pred = ridgereg.predict(X_test)

print("Coeficientes de Ridge con alpha = 0.1: ", '\n', '\n', ridgereg.coef_)
print('\n', "RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Coeficientes de Ridge con alpha = 0.1:  
 
 [ 2.44588347e-02 -1.23753983e+00  3.14879877e-02  2.99064342e-02
 -1.20249690e+00  4.40719004e-03 -1.38038043e-03 -3.77806705e+01
  2.22099107e-01  5.89285509e-01  2.62397115e-01  1.39215540e-01]

 RMSE:  0.7419468070835197


In [ ]:
# Regresión de Ridge con alpha = 1

ridgereg = Ridge(alpha=1, normalize=True)
ridgereg.fit(X_train, y_train)
y_pred = ridgereg.predict(X_test)

print("Coeficientes de Ridge con alpha = 1: ", '\n', '\n', ridgereg.coef_)
print('\n', "RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Coeficientes de Ridge con alpha = 1:  
 
 [ 1.86223905e-04 -5.68051730e-01  1.68619002e-01  5.73521034e-03
 -1.33305626e+00  1.61200670e-03 -5.81848853e-04 -2.30475660e+01
  7.65433089e-02  2.91695372e-01  1.40694735e-01 -1.26603805e-02]

 RMSE:  0.77294081168326


Analisis de los coeficientes:

1. fixed acidity: El aumento de una unidad de acidez fija implicaría un incremento de 0.000186 puntos en la calidad del vino.

2. volatile acidity: El aumento de una unidad en la acidez volátil implicaría una disminución de -0.568 puntos en la calidad del vino.

3. citric acid: El aumento de una unidad en el ácido cítrico implicaría un aumento del 0.1686 puntos en la calidad del vino.

4. residual sugar: El aumento de una unidad del azúcar residual implicaría un incremento de 0.005735 puntos en la calidad del vino.

5. chlorides: El aumento de una unidad en las calorias implicaría una disminución de -1.333 puntos en la calidad del vino

6. free sulfur dioxide: El aumento de una unidad del dióxido de azufre libre implicaría un incremento de 0.001612 puntos en la calidad del vino.

7. total sulfur dioxide: El aumento de una unidad del dióxido de azufre total implicaría una disminución de 0.0005818 puntos en la calidad del vino

8. density: El aumento de una unidad de la densidad implicaría una disminución de 0.2304 puntos en la calidad del vino

9. pH: El aumento de una unidad del pH implicaría un incremento de 0.1406 puntos en la calidad del vino.

10. sulphates: El aumento de una unidad del sulfato implicaría una disminución de 0.01266 puntos en la calidad del vino.

11. alcohol: El aumento de una unidad de alcohol implicaría un incremento de 0.2320 puntos en la calidad del vino.

12. type: Entre mas se acerque al tipo de vino rojo en una unidad, la calidad del vino aumenta en 0.3848

# Ejercicio 4

* Estimar una regresión lasso con alfa igual a 0,01, 0,1 y 1.
* Comparar los coeficientes con la regresión lineal
* Evaluar el RMSE

In [ ]:
# Lasso con alpha = 0.01

lassoreg1 = Lasso(alpha=0.01, normalize=True)
lassoreg1.fit(X_train, y_train)
y_pred = lassoreg1.predict(X_test)

print("Coeficientes de Lasso con alpha = 0.01: ", lassoreg1.coef_)
print('\n', "RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Coeficientes de Lasso con alpha = 0.01:  [-0. -0.  0. -0. -0.  0. -0. -0.  0.  0.  0. -0.]

 RMSE:  0.8683174491433667


In [ ]:
# Lasso con alpha = 0.1

lassoreg2 = Lasso(alpha=0.1, normalize=True)
lassoreg2.fit(X_train, y_train)
y_pred = lassoreg2.predict(X_test)

print("Coeficientes de Lasso con alpha = 0.1: ", lassoreg2.coef_)
print('\n', "RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Coeficientes de Lasso con alpha = 0.1:  [-0. -0.  0. -0. -0.  0. -0. -0.  0.  0.  0. -0.]

 RMSE:  0.8683174491433667


In [ ]:
# Lasso con alpha = 1

from sklearn.linear_model import Lasso
lassoreg3 = Lasso(alpha=1, normalize=True)
lassoreg3.fit(X_train, y_train)
y_pred = lassoreg3.predict(X_test)

print("Coeficientes de Lasso con alpha = 1: ", lassoreg3.coef_)
print('\n', "RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Coeficientes de Lasso con alpha = 1:  [-0. -0.  0. -0. -0.  0. -0. -0.  0.  0.  0. -0.]

 RMSE:  0.8683174491433667


Los coeficientes dados por la regresión de lasso fueron de cero, por otro lado los calculados en las regresiones anteriores dieron resultados distintos a cero.

A su vez, los RMSE de lasso han sido los más altos.




# Ejercicio 5

* Estandarizar las características (excepto la calidad del vino)

* Crear una variable objetivo binaria para cada tipo de vino

* Analizar los coeficientes

* * Evalúe con F1, AUC-ROC y log-loss

In [ ]:
feature_cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide',
                 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol','type2']

X = data[feature_cols]
y = data.quality2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=456)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) 

(5197, 12) (1300, 12) (5197,) (1300,)


In [ ]:
#Estandarizar las variables
sc = StandardScaler()
X_train2 = sc.fit_transform(X_train)
X_test2 = sc.transform(X_test)

# Crear variables binarias
data['type2'] = data['type'].map({'white':0, 'red':1})
data['quality2'] = data.quality.map({3:0, 4:0, 5:0, 6:1, 7:1, 8:1, 9:1})

In [ ]:
#Regresion logistica:
logreg = LogisticRegression(solver='liblinear',C=1e9)
logreg.fit(X_train2, y_train)
y_pred = logreg.predict(X_test2)

print(logreg.coef_)

[[ 0.12271096 -0.81606621 -0.06079562  0.56870553 -0.04899546  0.29175436
  -0.31925948 -0.38614463  0.1286662   0.30265429  1.02791489  0.31196008]]


In [ ]:
#F1
print('F1 score:', metrics.f1_score(y_test, y_pred))

F1 score: 0.7777777777777777


In [ ]:
#AUC-ROC
print('ROC AUC score:', metrics.roc_auc_score(y_test, y_pred, average=None))

ROC AUC score: 0.683875


In [ ]:
#Log-loss
print('Log Loss:', metrics.log_loss(y_test, y_pred))

Log Loss: 9.88354086856439


# Ejercicio 6

* Estimar una regresión logística regularizada usando:
* C = 0.01, 0.1 & 1.0
* penalización = ['l1, 'l2']
* Compare los coeficientes y la puntuación f1.

Nota: Para los valores de C y Penalización, deben realizar todas las posibles combinaciones entre estas dos variables.

**6.1) Penalización l1**

In [ ]:
# Regresión logistica regularizada con C = 0.01
logreg = LogisticRegression(C=0.01, penalty='l1',solver='liblinear',multi_class='auto')
logreg.fit(X_train2, y_train)
y_pred_prob = logreg.predict_proba(X_test2)

print(logreg.coef_)
print(metrics.log_loss(y_test, y_pred_prob))

[[ 0.00000000e+00 -5.13420274e-01  0.00000000e+00  7.39450739e-02
   0.00000000e+00  3.48526937e-05  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.33033691e-01  9.11121219e-01  0.00000000e+00]]
0.5569740754320911


In [ ]:
# Regresión logistica regularizada con C = 0.1
logreg = LogisticRegression(C=0.1, penalty='l1',solver='liblinear',multi_class='auto')
logreg.fit(X_train2, y_train)
y_pred_prob = logreg.predict_proba(X_test2)

print(logreg.coef_)
print(metrics.log_loss(y_test, y_pred_prob))

[[ 0.         -0.7824567  -0.04923653  0.32049854 -0.04396018  0.27073353
  -0.31027829 -0.02853623  0.05150993  0.26342708  1.14017328  0.16025228]]
0.5497026030032377


In [ ]:
# Regresión logistica regularizada con C = 1
logreg = LogisticRegression(C=1, penalty='l1',solver='liblinear',multi_class='auto')
logreg.fit(X_train2, y_train)
y_pred_prob = logreg.predict_proba(X_test2)

print(logreg.coef_)
print(metrics.log_loss(y_test, y_pred_prob))

[[ 0.10416063 -0.8131621  -0.058925    0.53746497 -0.04901486  0.28974405
  -0.31860711 -0.34065909  0.11777473  0.29820763  1.04279947  0.29586063]]
0.5500223467562161


**6.2) Penalización l2**

In [ ]:
# Regresión logistica regularizada con C = 0.01

logreg = LogisticRegression(C=0.01, penalty='l2',multi_class='auto',solver='liblinear')
logreg.fit(X_train2, y_train)
y_pred_prob = logreg.predict_proba(X_test2)

print(logreg.coef_)
print(metrics.log_loss(y_test, y_pred_prob))

[[ 0.05370877 -0.61333693 -0.00984039  0.32820389 -0.08361312  0.22478727
  -0.27459385 -0.20399633  0.0873781   0.25543402  0.85594632  0.11346822]]
0.5481535312005067


In [ ]:
# Regresión logistica regularizada con C = 0.1

logreg = LogisticRegression(C=0.1, penalty='l2',multi_class='auto',solver='liblinear')
logreg.fit(X_train2, y_train)
y_pred_prob = logreg.predict_proba(X_test2)

print(logreg.coef_)
print(metrics.log_loss(y_test, y_pred_prob))

[[ 0.09917984 -0.78618167 -0.0539936   0.5079031  -0.05516917  0.28591875
  -0.31992243 -0.31846313  0.11479992  0.29530076  1.01984053  0.2636805 ]]
0.549266550529634


In [ ]:
# Regresión logistica regularizada con C = 1

logreg = LogisticRegression(C=1, penalty='l2',multi_class='auto',solver='liblinear')
logreg.fit(X_train2, y_train)
y_pred_prob = logreg.predict_proba(X_test2)

print(logreg.coef_)
print(metrics.log_loss(y_test, y_pred_prob))

[[ 0.11947067 -0.81295874 -0.06011489  0.56094334 -0.04969763  0.29128744
  -0.31956177 -0.3768716   0.12674639  0.3017911   1.02794819  0.30608384]]
0.550017503357314


# SVM

# Ejercicio 7

Teniendo en cuenta las mismas variables de la regresión logística:

* Crear un objetivo binario para cada tipo de vino
* Crear dos SVM lineales para los vinos blancos y tintos, respectivamente.


In [ ]:
#Train and test Data

data_ww = data[data['type']=='white']
data_rw = data[data['type']!='white']

#Definir variables
xw = data_ww.drop(columns=['quality','type','quality2','type2'])
yw = data_ww.quality

xr = data_rw.drop(columns=['quality','type','quality2','type2'])
yr = data_rw.quality

#Partición de sets
X1_train, X1_test, y1_train, y1_test = train_test_split(xw, yw, test_size=0.2, random_state=456)
X2_train, X2_test, y2_train, y2_test = train_test_split(xr, yr, test_size=0.2, random_state=456)


In [ ]:
# MODELO VINO BLANCO
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

clf1 = SVC(kernel='linear')
clf1.fit(X1_train, y1_train)

clf1.score(X1_test,y1_test)

0.49795918367346936

In [ ]:
# MODELO VINO TINTO 

clf = SVC(kernel='linear')
clf.fit(X2_train, y2_train)

clf.score(X2_test, y2_test)

0.6125

# Ejercicio 8

* Probar las dos SVM utilizando los diferentes kernels ('poly', 'rbf', 'sigmoid')
* Evalúe con F1, AUC-ROC y log-loss


In [ ]:
#Modelo 1: Vino Blanco

wclf_poly = SVC(kernel='poly')
wclf_poly.fit(X1_train, y1_train)

wclf_rbf = SVC(kernel='rbf')
wclf_rbf.fit(X1_train, y1_train)

wclf_sigmoid = SVC(kernel='sigmoid')
wclf_sigmoid.fit(X1_train, y1_train)

print('Kernel Poly: ', wclf_poly.score(X1_test, y1_test))
print('Kernel rbf: ', wclf_rbf.score(X1_test, y1_test))
print('Kernel sigmoid: ', wclf_sigmoid.score(X1_test, y1_test))

Kernel Poly:  0.45714285714285713
Kernel rbf:  0.4561224489795918
Kernel sigmoid:  0.4336734693877551


In [ ]:
#Modelo 2: Vino Tinto

rclf_poly = SVC(kernel='poly')
rclf_poly.fit(X2_train, y2_train)

rclf_rbf = SVC(kernel='rbf')
rclf_rbf.fit(X2_train, y2_train)

rclf_sigmoid = SVC(kernel='sigmoid')
rclf_sigmoid.fit(X2_train, y2_train)

print('Kernel Poly: ', rclf_poly.score(X2_test, y2_test))
print('Kernel rbf: ', rclf_rbf.score(X2_test, y2_test))
print('Kernel sigmoid: ', rclf_sigmoid.score(X2_test, y2_test))

Kernel Poly:  0.4625
Kernel rbf:  0.509375
Kernel sigmoid:  0.475


# Ejercicio 9
* Utilizando el mejor SVM encontrar los parámetros que da el mejor rendimiento teniendo en cuenta los siguientes hiperparámetros
'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]
* Evalúe cada modelo con F1, AUC-ROC y log-loss

Nota: Para los valores de C y gamma, deben realizar todas las posibles combinaciones entre estas dos variables.

In [ ]:
#Para el modelo 1: Vino Blanco

#svc_clf = svm.SVC()
# Grilla de validación
#parameters = {'gamma': np.linspace(0.01, 0.001, 0.0001), 'C': np.linspace(0.1, 1, 10, 100, 1000)}
#acc_scorer = make_scorer(roc_auc_score)

# Run and fit the grid search
#grid_obj = GridSearchCV(svc_clf, parameters, scoring=acc_scorer)
#grid_obj = grid_obj.fit(X_train, y_train)

parameters_w = [{'C': [0.1,1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.01, 0.001, 0.0001]}]

gs_w = GridSearchCV(estimator = rclf_rbf, param_grid = parameters_w,
                           scoring = 'accuracy', cv = 10,)

gs_w.fit(X1_train, y1_train)
best_parameters_w = gs_w.best_params_

#best_accuracy_w = gs_w.best_score_
#best_accuracy_w

#mejores parametros
print(best_parameters_w)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}


In [ ]:
#Para el modelo 2: Vino Tinto

parameters_r = [{'C': [0.1,1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.01, 0.001, 0.0001]}]

gs_r = GridSearchCV(estimator = rclf_rbf, param_grid = parameters_r, scoring = 'accuracy', cv = 10,)

gs_r.fit(X2_train, y2_train)
best_parameters_r = gs_r.best_params_

#best_accuracy_r = gs_r.best_score_
#best_accuracy_r

#Mejores parametros:
best_parameters_r

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

# Ejercicio 10

Compare los resultados con todos los anteriores modelos y eliga ¿Cuál algoritmo con cuáles hiperparámetros es el que tiene mejor rendimiento según la puntuación F1, AUC-ROC y log-loss?

De los mejores modelos quedó: Regresión logistica regularizada con C = 0.01, el con un log-loss de: 0.5481